<a href="https://colab.research.google.com/github/Muppidathe/deep_learning/blob/main/ann_torch(regression).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder,StandardScaler,OneHotEncoder,MinMaxScaler
import pickle
import numpy as np

In [2]:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# data transformation


In [5]:
#drop unwanted column
df=df.drop(columns=['RowNumber','CustomerId','Surname'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
print(df['Gender'].unique())
gender_encoder=LabelEncoder()
df['Gender']=gender_encoder.fit_transform(df['Gender'])
df.head()

['Female' 'Male']


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [7]:
print(df['Geography'].unique())
print('len: ',len(df['Geography'].unique()))
geo_encoder=OneHotEncoder()
temp_df=geo_encoder.fit_transform(df[['Geography']]).toarray()
temp_df=pd.DataFrame(temp_df,columns=geo_encoder.get_feature_names_out())
temp_df.head()

['France' 'Spain' 'Germany']
len:  3


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [8]:
x=pd.concat([temp_df,df.drop(columns=['Geography','EstimatedSalary'])],axis=1)
y=df['EstimatedSalary']
x.shape

(10000, 12)

# train test spliting

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=42)
x_train.shape,y_train.shape

((8000, 12), (8000,))

#  scaling all the feature using minmaxscaler

In [36]:
scaler=StandardScaler()
x_train_transformed=scaler.fit_transform(x_train)
x_test_transformed=scaler.transform(x_test)
x_transformed[:2,:]

array([[1.        , 0.        , 0.        , 0.672     , 1.        ,
        0.18918919, 0.6       , 0.        , 0.33333333, 1.        ,
        1.        , 0.        ],
       [0.        , 1.        , 0.        , 0.564     , 1.        ,
        0.32432432, 0.4       , 0.47678561, 0.33333333, 1.        ,
        1.        , 0.        ]])

# designing model and compiling

In [42]:
class mymodel(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1=nn.Linear(12,32)
    self.fc2=nn.Linear(32,16)
    self.fc3=nn.Linear(16,5)
    self.fc4=nn.Linear(5,1)
  def forward(self,x):
    x=nn.functional.relu(self.fc1(x))
    x=nn.functional.relu(self.fc2(x))
    x=nn.functional.relu(self.fc3(x))
    x=self.fc4(x)
    return x

model=mymodel()
critirion=nn.SmoothL1Loss()
opti=torch.optim.Adam(model.parameters(),lr=.01)

# training model

In [41]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
epochs=100
train_batch_size=1024
test_batch_size=512
for epoch in range(0,epochs):
  train_true=[]
  test_true=[]
  train_pred=[]
  test_pred=[]
  for i in range(0,len(x_train_transformed),train_batch_size):
    x_batch=torch.tensor(x_train_transformed[i:i+train_batch_size,],dtype=torch.float32)
    y_batch=torch.tensor(np.array(y_train[i:i+train_batch_size,]).reshape(-1,1),dtype=torch.float32)
    opti.zero_grad()
    outputs=model(x_batch)
    loss=critirion(outputs,y_batch)
    loss.backward()
    opti.step()

    train_pred.extend(outputs.detach().numpy().flatten())
    train_true.extend(y_batch.detach().numpy().flatten())
  model.eval()
  with torch.no_grad():
    for i in range(0,len(x_test_transformed),test_batch_size):
      x_batch=torch.tensor(x_test_transformed[i:i+test_batch_size,],dtype=torch.float32)
      y_batch=torch.tensor(np.array(y_test[i:i+test_batch_size,]).reshape(-1,1))
      outputs=model(x_batch)
      loss=critirion(outputs,y_batch)
      test_pred.extend(outputs.detach().numpy().flatten())
      test_true.extend(y_batch.detach().numpy().flatten())
  model.train()
  #loss
  train_loss=mean_squared_error(train_true,train_pred)
  test_loss=mean_squared_error(test_true,test_pred)
  #mae
  train_mae=mean_absolute_error(train_true,train_pred)
  test_mae=mean_absolute_error(test_true,test_pred)
  #r2
  train_r2=r2_score(train_true,train_pred)
  test_r2=r2_score(test_true,test_pred)
  print(f'epoch [{epoch+1}/{epochs}] train_loss {train_loss} train_mean_abs_error {train_mae} train_r2_score {train_r2} val_loss {test_loss} val_mean_abs_error {test_mae} val_r2_score {test_r2}')
  print('-'*100)

epoch [1/100] train_loss 13394319846.122828 train_mean_abs_error 100430.61439748222 train_r2_score -3.049053386531158 val_loss 13047792570.09823 val_mean_abs_error 98724.56291110789 val_r2_score -2.9523755025997946
----------------------------------------------------------------------------------------------------
epoch [2/100] train_loss 13393814532.395933 train_mean_abs_error 100428.12597074067 train_r2_score -3.0489006320590644 val_loss 13046747629.38222 val_mean_abs_error 98719.25954018364 val_r2_score -2.952058974109213
----------------------------------------------------------------------------------------------------
epoch [3/100] train_loss 13391853472.26369 train_mean_abs_error 100418.45078176283 train_r2_score -3.048307810828806 val_loss 13043203604.608992 val_mean_abs_error 98701.27066789963 val_r2_score -2.9509854349171194
----------------------------------------------------------------------------------------------------
epoch [4/100] train_loss 13385841675.692596 train_me

In [ ]:
model.predict(x_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


array([[ 95566.73],
       [ 94029.27],
       [ 96705.62],
       ...,
       [101620.33],
       [ 93681.64],
       [ 93032.64]], dtype=float32)